<a href="https://colab.research.google.com/github/Monish-Kumar-D/Email_spam_detector/blob/main/99_9_score_on_train_set_and_97_8_on_test_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'email-spam-classification-dataset-csv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F547699%2F998616%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240714%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240714T072547Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7574a50b9228697b97d74c45dfaa4a2e36b884c750934132d4c9c5f3c2016dd79a1c1d807763934aea8450235c0dd14b2acf1bc4573a3e7a4c285ea98d1d56c3811d81f15bd9994a74a50b76fa4a075a1710b03e876ab2aabe51a944317ce3ed6b1130e8084faeea1ed53d6ce1bdc56d551434f357af307061eb7b6211dea1b1b51a42e5927044ef4f96fb19901ab30ac4a1b608babef8267e22de16bae302fb8546b6420e10e30b962129f2efcfd1889c35c82dc41bb0486260804e01df2886f7d14f9880d52b9d9d35766e608d0cab1c436eb1672f203cf69a29d2235047f9335046ae6bded8739ae035fa9f5c4698f25258e885097d24291d78f5b7702c59'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1738291 bytes downloaded
Downloaded and uncompressed: email-spam-classification-dataset-csv
Data source import complete.


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/email-spam-classification-dataset-csv/emails.csv


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv('/kaggle/input/email-spam-classification-dataset-csv/emails.csv')

In [5]:
df.head()

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [7]:
df.sample()

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
4585,Email 4586,9,4,2,0,5,2,40,3,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
input_X = df.drop(columns=['Prediction', 'Email No.'])
input_Y = df.Prediction

In [9]:
input_X.head(1)

,the,to,ect,and,for,of,a,you,hou,in,...,enhancements,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry
0,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
input_Y.head(1)

0    0
Name: Prediction, dtype: int64

In [11]:
scaler = StandardScaler()
X = scaler.fit_transform(input_X)

In [12]:
X_train, x_test, Y_train, y_test = train_test_split(X, input_Y, test_size=0.3, random_state=42)

In [13]:
model = LogisticRegression(max_iter = 300, C=1.5)
model.fit(X_train,Y_train)

LogisticRegression(C=1.5, max_iter=300)

In [14]:
y_pred = model.predict(x_test)

In [15]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy*100)

96.52061855670104


In [16]:
from xgboost import XGBClassifier

In [17]:
xbg_model = XGBClassifier(n_estimators = 300, max_depth = 4)

In [18]:
xbg_model.fit(
    X_train, Y_train,
    eval_set = [(x_test, y_test)],
    verbose = 1,
    early_stopping_rounds = 5

)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.48575
[1]	validation_0-logloss:0.40823
[2]	validation_0-logloss:0.35706
[3]	validation_0-logloss:0.32153
[4]	validation_0-logloss:0.29355
[5]	validation_0-logloss:0.26966
[6]	validation_0-logloss:0.24989
[7]	validation_0-logloss:0.23451
[8]	validation_0-logloss:0.22012
[9]	validation_0-logloss:0.20877
[10]	validation_0-logloss:0.19727
[11]	validation_0-logloss:0.18704
[12]	validation_0-logloss:0.18007
[13]	validation_0-logloss:0.17207
[14]	validation_0-logloss:0.16609
[15]	validation_0-logloss:0.16080
[16]	validation_0-logloss:0.15529
[17]	validation_0-logloss:0.15009
[18]	validation_0-logloss:0.14587
[19]	validation_0-logloss:0.14274
[20]	validation_0-logloss:0.13759
[21]	validation_0-logloss:0.13376
[22]	validation_0-logloss:0.13085
[23]	validation_0-logloss:0.12871
[24]	validation_0-logloss:0.12547
[25]	validation_0-logloss:0.12184
[26]	validation_0-logloss:0.11924
[27]	validation_0-logloss:0.11732
[28]	validation_0-logloss:0.11568
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [19]:
xbg_model.score(X_train, Y_train)

0.9994475138121547

In [20]:
xbg_model.score(x_test, y_test)

0.9780927835051546